In [1]:
include("../src/iTEBD.jl")
using Main.iTEBD, LinearAlgebra, Test

### Test AKLT

In [2]:
aklt = begin
    aklt_tensor = zeros(2,3,2)
    aklt_tensor[1,1,2] = +sqrt(2/3)
    aklt_tensor[1,2,1] = -sqrt(1/3)
    aklt_tensor[2,2,2] = +sqrt(1/3)
    aklt_tensor[2,3,1] = -sqrt(2/3)
    aklt_tensor
    iMPS([aklt_tensor, aklt_tensor])
end

@test aklt.λ[1] ≈ fill(sqrt(2)/2, 2)
@test aklt.λ[2] ≈ fill(sqrt(2)/2, 2)

Test Passed

#### Test left and right canonical form

In [13]:
Γ1, λ1 = aklt[1]
Γ2, λ2 = aklt[2]

Γ = iTEBD.tensor_rmul!(deepcopy(Γ1), λ1)
M = reshape(Γ, 2, 6)
@test norm(M * M' - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_rmul!(deepcopy(Γ2), λ2)
M = reshape(Γ, 2, 6)
@test norm(M * M' - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_lmul!(λ2, deepcopy(Γ1))
M = reshape(Γ, 6, 2)
@test norm(M' * M - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_lmul!(λ1, deepcopy(Γ2))
M = reshape(Γ, 6, 2)
@test norm(M' * M - I) ≈ 0 atol = 1e-12

Test Passed

### Test random tensor

In [25]:
A = randn(ComplexF64, 3, 3, 3)
B = randn(ComplexF64, 3, 3, 3)
ψ = iMPS([A, B])
ψ.λ

2-element Vector{Vector{Float64}}:
 [0.6682168843749184, 0.5462153018123038, 0.5051089382523047]
 [0.7475051412261391, 0.510536837273602, 0.4249567055914725]

In [26]:
ψ2 = iMPS([A,B], renormalize=false)
n2 = inner_product(ψ2, ψ2)
c2 = inner_product(ψ2, ψ)
@test n2 - c2^2 ≈ 0 atol=1e-8

Test Passed

In [27]:
Γ1, λ1 = ψ[1]
Γ2, λ2 = ψ[2]

Γ = iTEBD.tensor_rmul!(deepcopy(Γ1), λ1)
M = reshape(Γ, 3, 9)
@test norm(M * M' - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_rmul!(deepcopy(Γ2), λ2)
M = reshape(Γ, 3, 9)
@test norm(M * M' - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_lmul!(λ2, deepcopy(Γ1))
M = reshape(Γ, 9, 3)
@test norm(M' * M - I) ≈ 0 atol = 1e-12

Γ = iTEBD.tensor_lmul!(λ1, deepcopy(Γ2))
M = reshape(Γ, 9, 3)
@test norm(M' * M - I) ≈ 0 atol = 1e-12

Test Passed

### Dimension redundant

In [28]:
A = zeros(ComplexF64, 4, 3, 4)
B = zeros(ComplexF64, 4, 3, 4)
A[1:3, :, 1:3] = randn(ComplexF64, 3, 3, 3)
B[1:3, :, 1:3] = randn(ComplexF64, 3, 3, 3)

h1 = randn(ComplexF64, 4, 4) |> Hermitian |> Array
h2 = randn(ComplexF64, 4, 4) |> Hermitian |> Array
U1, U2 = exp(-1im*h1), exp(-1im*h1)

A = iTEBD.canonical_gauging(A, U1, U2')
B = iTEBD.canonical_gauging(B, U2, U1')

ψ = iMPS([A, B])
ψ.λ

2-element Vector{Vector{Float64}}:
 [0.824736000306301, 0.5119106099867542, 0.24032906021069134]
 [0.8821550954884595, 0.4110792550581315, 0.22981782690774635]

In [29]:
ψ2 = iMPS([A,B], renormalize=false)
n2 = inner_product(ψ2, ψ2)
c2 = inner_product(ψ2, ψ)
@test n2 - c2^2 ≈ 0 atol=1e-8

Test Passed